In [ ]:
#初期設定
import os
import pandas as pd
import numpy as np
import time
import lxml
import re
import requests
from bs4 import BeautifulSoup as bs4
import tqdm

In [ ]:
# https://web-kiwami.com/python-beautyfulsoup4.html
# http://kondou.com/BS4/
# bs4参考

In [ ]:
# all_year
this_year = 2022 #年変わったら変更
get_year = list(range(2011,this_year+1))
df_result = pd.DataFrame()
dict1 = {}
for year in get_year:
    if 2011 <= year <= 2017:
        print(year)
        # 単年のデータ取得2011~2017
        url =  rf"https://www.ipokiso.com/company/{year}.html"
        html_res = requests.get(url)
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        df_url = pd.read_html(url)
        time.sleep(np.random.randint(100,120)/100)
        for i in range(len(df_url)):
            df_url[i]["上場年"] = f"{year}"
            df_result = pd.concat([df_result, df_url[i]])
            df_result = df_result.reset_index(drop=True)
        # df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
        # df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
    else:
        # 単年のデータ取得2018~2022
        print(year)
        url = rf"https://www.ipokiso.com/company/{year}.html"
        if year == 2022:
            url = r"https://www.ipokiso.com/company/index.html"
        html_res = requests.get(url)
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        df_url = pd.read_html(url)
        time.sleep(np.random.randint(100,120)/100)
        for i in range(0, len(df_url) , 2):
            df_url_con = pd.concat([df_url[i], df_url[i+1]], axis=1)
            df_url_con["上場年"] = f"{year}"
            if year == 2019:
                df_url_con = df_url_con.rename(columns={'上場 市場': '上場市場'})
            if year >= 2020:
                df_url_con = df_url_con.rename(columns={'申し込み期間': '申し込み 期間', '初値上昇率': '初値 上昇率'})
            df_result = pd.concat([df_result, df_url_con],ignore_index=True)
            df_result = df_result.reset_index(drop=True)
        df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
        df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
    if year != this_year:
        time.sleep(np.random.randint(100,120)/100)

In [ ]:
# dict作成
# 空白削除
del dict1[" "]
del dict1[""]
del dict1["\n"]
# dictをdf化
df_dict = pd.DataFrame.from_dict(dict1, orient='index')
# カラム名変更
df_dict = df_dict.rename(columns={0:"url"})
# 間違えているurlを変更
df_dict.url[df_dict.url == "https://www.ipokiso.com/https://www.ipokiso.com/company/2013/zigexn.html"] = "https://www.ipokiso.com/company/2013/zigexn.html"
df_dict = df_dict.rename_axis('index').reset_index()
df_dict = df_dict.rename(columns={'index':'company_name'})

In [ ]:
df_dict

In [ ]:
# 時短のためcode含めたcsv読み込み
df_result2 = pd.read_csv("df_dict.csv", index_col=0)
df_result2

In [1132]:
# code_listの作成（詳細情報の取得ループ 全てのcode取得[0]までok！
def make_company_info(url):
    html_res = requests.get(url)
    time.sleep(np.random.randint(100,120)/100)
    soup = bs4(html_res.content, 'html.parser')
    # 全企業codeの取得
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    code_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        val_list = []
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            else:
                keyword = np.nan
            val_list.append(keyword)
    except AttributeError:
        val_list = [np.nan,np.nan,np.nan,np.nan]
        print(f"成長性listのエラー{code}")
    evaluation_list.append(val_list)
    print(code)
code_list = []
url_list = df_dict.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[0:3]] #ここで検証のurl数変更！
df_company_01 = pd.DataFrame(evaluation_list,columns=["成長性","割安性","話題性","総合評価"])
df_result2[df_company_01.columns] = df_company_01
print(df_result2)


3170
3169
9206
                company_name  \
0                     アイセイ薬局   
1                        ミサワ   
2                   スターフライヤー   
3                     新田ゼラチン   
4          カイオム・\r\nバイオサイエンス   
...                      ...   
1009               エッジテクノロジー   
1010                 ライトワークス   
1011   ビッグツリーテクノロジー＆コンサルティング   
1012                  セイファート   
1013  Recovery International   

                                                    url  code  成長性  割安性  話題性  \
0      https://www.ipokiso.com//company/2011/aisei.html  3170    ◎    〇    〇   
1      https://www.ipokiso.com//company/2011/unico.html  3169    〇    △    〇   
2     https://www.ipokiso.com//company/2011/starflye...  9206    〇    ◎    〇   
3     https://www.ipokiso.com//company/2011/nitta-ge...  4977  NaN  NaN  NaN   
4     https://www.ipokiso.com//company/2011/chiome.html  4583  NaN  NaN  NaN   
...                                                 ...   ...  ...  ...  ...   
1009  https://www.ipokiso.com//company/2

In [ ]:
# 20221201 ループで全table、全銘柄取り出す。（細かいデータの整形は後でやる）
def make_company_info_2(url):
    df_html_table = pd.read_html(url)
    for i in range(0,8):

        # 基本情報
        if i == 0:
            df_table = df_html_table[i + 1]
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = df_html_table[i + 2]
            # 基本情報のcols取得
            cols = df_table.T.values[0]
            # 基本情報のval取得
            val = df_table.T.values[1]
            # 追加する1件分の基本情報のdf作成
            df_table_1 = pd.DataFrame([val],columns=cols)
        # IPO日程と価格決定（初値予想）
        if i == 1:
            df_table = df_html_table[i + 1]
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = df_html_table[i + 2]
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_2 = pd.DataFrame([val],columns=cols)
        # 次ここから
        # if i == 2:

        # 全件のdfに追加していく
    df_table_all = pd.concat([df_table_1,df_table_2],axis=1)
    
    return df_table_all
df_all = pd.DataFrame()
url_list = df_dict.url.values
for url in url_list[0:3]:
    df_table_all = make_company_info_2(url) #ここでurl数変更
    df_all = pd.concat([df_all,df_table_all],axis=0)
print(df_all)

以上本番用

In [ ]:
""" 
# csv変換 時短のためこれを読み込む
df_code = pd.DataFrame(code_list,columns=["code"])
df_dict[df_code.columns] = df_code.values
df_result2.to_csv("df_result2", encoding='utf-8_sig')
"""

In [ ]:
# 時短のためcode含めたcsv読み込み
df_result2 = pd.read_csv("df_dict.csv", index_col=0)
df_result2

- IPO当選株数のデータ取得

In [ ]:
# 以下データ成長性等取得する
url = "https://www.ipokiso.com//company/2019/washingtonhotel.html"
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')

In [ ]:
df_html_table_2011 = pd.read_html("https://www.ipokiso.com//company/2019/washingtonhotel.html")[1]
df_html_table_2011

In [1012]:
df_html_table = pd.read_html("https://www.ipokiso.com/company/2022/tms-japan.html")
df_html_table

[   成長性   割安性
 0  NaN   NaN
 1  話題性  総合評価
 2  NaN   NaN,
          0                                                  1
 0      会社名                                ティムス（4891）　【東証グロース】
 1    会社URL                       https://www.tms-japan.co.jp/
 2     会社設立                                      2005年2月（18年目）
 3  狙い目証券会社  SMBC日興証券（主幹事）、野村證券（副幹事）、大和証券、SBI証券、楽天証券、岡三オンライ...,
           0                         1
 0      想定価格                      670円
 1       仮条件               640円 ～ 670円
 2      公募価格                      670円
 3  初値予想（独自）     616円 ～ 800円（10月18日時点）
 4        初値  919円　（公募価格比+249円　+37.2％）,
         0                    1
 0  抽選申込期間   11月7日(月)～11月11日(金)
 1   当選発表日            11月14日(月)
 2  購入申込期間  11月15日(火)～11月18日(金)
 3     上場日            11月22日(火),
             0                                         1
 0        公募株数   3,432,800株（国内：1,302,200株 海外2,130,600 株）
 1  売出株数（OA含む）          858,100株（国内：646,400株 海外：21,170株）
 2      当選株数合計  4,290,900株（国内：1,948,600株 海外：2,342,300 株）,
  

[0]の取得

In [ ]:
df_company_01 = pd.DataFrame(evaluation_list,columns=["成長性","割安性","話題性","総合評価"])
df_result2[df_company_01.columns] = df_company_01
print(df_result2)

In [1131]:
# code_listの作成（詳細情報の取得ループ 全てのcode取得[0]までok！
def make_company_info(url):
    html_res = requests.get(url)
    time.sleep(np.random.randint(100,120)/100)
    soup = bs4(html_res.content, 'html.parser')
    # 全企業codeの取得
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    code_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        val_list = []
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            else:
                keyword = np.nan
            val_list.append(keyword)
    except AttributeError:
        val_list = [np.nan,np.nan,np.nan,np.nan]
        print(f"成長性listのエラー{code}")
    evaluation_list.append(val_list)
    print(code)
code_list = []
url_list = df_dict.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[0:3]] #ここで検証のurl数変更！
df_company_01 = pd.DataFrame(evaluation_list,columns=["成長性","割安性","話題性","総合評価"])
df_result2[df_company_01.columns] = df_company_01
print(df_result2)


3170
3169
9206
                company_name  \
0                     アイセイ薬局   
1                        ミサワ   
2                   スターフライヤー   
3                     新田ゼラチン   
4          カイオム・\r\nバイオサイエンス   
...                      ...   
1009               エッジテクノロジー   
1010                 ライトワークス   
1011   ビッグツリーテクノロジー＆コンサルティング   
1012                  セイファート   
1013  Recovery International   

                                                    url  code  成長性  割安性  話題性  \
0      https://www.ipokiso.com//company/2011/aisei.html  3170    ◎    〇    〇   
1      https://www.ipokiso.com//company/2011/unico.html  3169    〇    △    〇   
2     https://www.ipokiso.com//company/2011/starflye...  9206    〇    ◎    〇   
3     https://www.ipokiso.com//company/2011/nitta-ge...  4977  NaN  NaN  NaN   
4     https://www.ipokiso.com//company/2011/chiome.html  4583  NaN  NaN  NaN   
...                                                 ...   ...  ...  ...  ...   
1009  https://www.ipokiso.com//company/2

In [ ]:
df_result2

In [ ]:
df_result2.to_csv("df_result2.csv", encoding='utf-8_sig')

全table取得（pd_htmlによる）

In [1016]:
df_html_table[2].T

,0,1,2,3,4
0,想定価格,仮条件,公募価格,初値予想（独自）,初値
1,670円,640円 ～ 670円,670円,616円 ～ 800円（10月18日時点）,919円 （公募価格比+249円 +37.2％）


In [1023]:
df_table_1

NameError: name 'df_table_1' is not defined

In [1102]:
cols = df_html_table[2].T.values[0]
# 基本情報のval取得
val = df_html_table[2].T.values[1]
# 追加する1件分の基本情報のdf作成
df_table_1 = pd.DataFrame([val],columns=cols)
df_table_1

,想定価格,仮条件,公募価格,初値予想（独自）,初値
0,670円,640円 ～ 670円,670円,616円 ～ 800円（10月18日時点）,919円 （公募価格比+249円 +37.2％）


In [1103]:
cols = df_html_table[3].T.values[0]
# 基本情報のval取得
val = df_html_table[3].T.values[1]
# 追加する1件分の基本情報のdf作成
df_table_2 = pd.DataFrame([val],columns=cols)
df_table_2

,抽選申込期間,当選発表日,購入申込期間,上場日
0,11月7日(月)～11月11日(金),11月14日(月),11月15日(火)～11月18日(金),11月22日(火)


In [1100]:
cols = df_html_table[4].T.values[0]
# 基本情報のval取得
val = df_html_table[4].T.values[1]
# 追加する1件分の基本情報のdf作成
df_table_3 = pd.DataFrame([val],columns=cols)
df_table_3

,公募株数,売出株数（OA含む）,当選株数合計
0,"3,432,800株（国内：1,302,200株 海外2,130,600 株）","858,100株（国内：646,400株 海外：21,170株）","4,290,900株（国内：1,948,600株 海外：2,342,300 株）"


In [1119]:
val[5]

'三菱UFJ  モルガン・  スタンレー証券'

In [1120]:
df_html_table[5].T.values[2]

array(['割当率', '85.77％', '2.13％', '2.13％', '2.13％', '2.13％', '2.13％',
       '2.13％', '1.07％', '0.35％', nan], dtype=object)

In [1108]:
cols = df_html_table[5].T.values[0]
# 基本情報のval取得
val = df_html_table[5].T.values[1]
# 追加する1件分の基本情報のdf作成
df_table_4 = pd.DataFrame([val],columns=cols)
df_table_4

,NaN,主幹事,幹事,幹事,幹事,幹事,幹事,幹事,幹事,幹事,幹事
0,証券会社名,SMBC日興証券,野村證券,みずほ証券,大和証券,三菱UFJ モルガン・ スタンレー証券,岡三証券,SBI証券,香川証券,楽天証券,NaN


In [1082]:
pd.concat([df_table_1,df_table_2],axis=0)

,想定価格,仮条件,公募価格,初値予想（独自）,初値,抽選申込期間,当選発表日,購入申込期間,上場日
0,670円,640円 ～ 670円,670円,616円 ～ 800円（10月18日時点）,919円 （公募価格比+249円 +37.2％）,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,11月7日(月)～11月11日(金),11月14日(月),11月15日(火)～11月18日(金),11月22日(火)


In [1084]:
df_all = pd.DataFrame()

In [1094]:
if df_all.empty == True:
    print("hi")

hi


In [1124]:
del df_all

In [1083]:
df_all = pd.concat([df_all,df_table_all],axis=0)

NameError: name 'df_all' is not defined

In [1059]:
df_table_all = pd.merge(df_table_1,df_table_2,on=["key"])

KeyError: 'key'

In [1129]:
# 20221201 ループで全table、全銘柄取り出す。（細かいデータの整形は後でやる）
def make_company_info(url):
    df_html_table = pd.read_html(url)
    for i in range(0,8):

        # 基本情報
        if i == 0:
            df_table = df_html_table[i + 1]
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = df_html_table[i + 2]
            # 基本情報のcols取得
            cols = df_table.T.values[0]
            # 基本情報のval取得
            val = df_table.T.values[1]
            # 追加する1件分の基本情報のdf作成
            df_table_1 = pd.DataFrame([val],columns=cols)
        # IPO日程と価格決定（初値予想）
        if i == 1:
            df_table = df_html_table[i + 1]
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = df_html_table[i + 2]
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_2 = pd.DataFrame([val],columns=cols)
        # 次ここから
        # if i == 2:

        # 全件のdfに追加していく
    df_table_all = pd.concat([df_table_1,df_table_2],axis=1)
    
    return df_table_all
df_all = pd.DataFrame()
code_list = []
url_list = df_dict.url.values
evaluation_list = []
for url in url_list[0:3]:
    df_table_all = make_company_info(url)
    df_all = pd.concat([df_all,df_table_all],axis=0)
# df = [make_company_info(url) for url in url_list[0:3]] #ここで検証のurl数変更！


[1]の取得

In [904]:
df_html_table_2011 = pd.read_html("https://www.ipokiso.com//company/2019/washingtonhotel.html")
cols = df_html_table_2011.T.values[0]
val = df_html_table_2011.T.values[1]
df_basic_info2 = pd.DataFrame([val],columns=cols)
df_basic_info2

AttributeError: 'list' object has no attribute 'T'

In [900]:
df_html_table_2011 = pd.read_html("https://www.ipokiso.com//company/2019/washingtonhotel.html")[1]
cols = df_html_table_2011.T.values[0]
val = df_html_table_2011.T.values[1]
df_basic_info = pd.DataFrame([val],columns=cols)
df_basic_info

,NaN,話題性,NaN
0,NaN,総合評価,NaN


In [913]:
df_html_table_2011 = pd.read_html("https://www.ipokiso.com//company/2019/washingtonhotel.html")

df_html_table_2011[2]

,0,1
0,会社名,ワシントンホテル（4691） 【東証2部、名証2部】
1,会社URL,https://www.washingtonhotel.co.jp
2,狙い目証券会社,東海東京証券、SMBC日興証券、大和証券、SBI証券、マネックス証券 岡三オンライン証券、カ...


In [989]:
# 時短のためcode含めたcsv読み込み
df_result2 = pd.read_csv("df_dict.csv", index_col=0)
df_result2

,company_name,url,code
0,アイセイ薬局,https://www.ipokiso.com//company/2011/aisei.html,3170
1,ミサワ,https://www.ipokiso.com//company/2011/unico.html,3169
2,スターフライヤー,https://www.ipokiso.com//company/2011/starflye...,9206
3,新田ゼラチン,https://www.ipokiso.com//company/2011/nitta-ge...,4977
4,カイオム・\r\nバイオサイエンス,https://www.ipokiso.com//company/2011/chiome.html,4583
...,...,...,...
1009,エッジテクノロジー,https://www.ipokiso.com//company/2022/edge-tec...,4268
1010,ライトワークス,https://www.ipokiso.com//company/2022/lightwor...,4267
1011,ビッグツリーテクノロジー＆コンサルティング,https://www.ipokiso.com//company/2022/bigtreet...,4266
1012,セイファート,https://www.ipokiso.com//company/2022/seyfert....,9213


In [998]:
# 202221130
def make_company_info(url):
    html_res = requests.get(url)
    time.sleep(np.random.randint(100,120)/100)
    soup = bs4(html_res.content, 'html.parser')
    # 全企業codeの取得
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    code_list.append(code)
    # table[0]成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        val_list = []
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            else:
                keyword = np.nan
            val_list.append(keyword)
    except AttributeError:
        val_list = [np.nan,np.nan,np.nan,np.nan]
        print(f"成長性listのエラー{code}")
    evaluation_list.append(val_list)

    # read_htmlでtable情報取得
    df_html_table = pd.read_html(url)

    #[1]基本情報
    # 関数外の変数が取れないためglobalで定義する
    global df_basic_info
    time.sleep(np.random.randint(100,120)/100)
    for i in range(1,8):
        df_table = df_html_table[1 + i].copy()
        # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
        if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
            df_table = df_html_table[2 + i].copy()
        # 基本情報のcols取得
        cols = df_table.T.values[0]
        # 基本情報のval取得
        val = df_table.T.values[1]
        # 追加する1件分の基本情報のdf作成
        df_table_new = pd.DataFrame([val],columns=cols)
        print(df_table_new)
        # 全件のdfに追加していく
        # df_basic_info = pd.merge([df_basic_info,df_table_new])
        return df_basic_info

    """
    # table基本情報のみ取取得
    df_table_basic = df_html_table[1].copy()
    # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
    if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
        df_table_basic = df_html_table[2].copy()
    # 基本情報のcols取得
    cols = df_table_basic.T.values[0]
    # 基本情報のval取得
    val = df_table_basic.T.values[1]
    # 追加する1件分の基本情報のdf作成
    df_basic_info_new = pd.DataFrame([val],columns=cols)
    # 全件のdfに追加していく
    df_basic_info = pd.concat([df_basic_info,df_basic_info_new],ignore_index=True)
    print(code)
    print(evaluation_list)
    return df_basic_info
    """
    
code_list = []
url_list = df_dict.url.values
evaluation_list = []


# 基本情報の空df作成
df_basic_info = pd.DataFrame([],columns=["会社名","コード","市場","会社URL","狙い目証券会社","会社設立"])
[make_company_info(url) for url in url_list[range(3)]] #ここで検証のurl数変更！
df_company_01 = pd.DataFrame(evaluation_list,columns=["成長性","割安性","話題性","総合評価"])
df_result2[df_company_01.columns] = df_company_01
#[1]データ分割、整理、結合　codeの場所を分ける？ 
# 結合(axis=1で横方向の結合)
df_result2 = pd.concat([df_result2,df_basic_info],axis=1)


print(df_result2)

     想定価格              仮条件    公募価格 初値予想（独自）                 初値
0  3,460円  3,300円 ～ 3,460円  3,460円        -  3,460円（公募価格比 ±0％）
   想定価格          仮条件  公募価格 初値予想（独自）                    初値
0  860円  760円 ～ 860円  860円        -  1,209円（公募価格比 +40.6％）
     想定価格              仮条件    公募価格 初値予想（独自）                    初値
0  3,180円  2,900円 ～ 3,180円  3,180円        -  5,500円（公募価格比 +73.0％）
                company_name  \
0                     アイセイ薬局   
1                        ミサワ   
2                   スターフライヤー   
3                     新田ゼラチン   
4          カイオム・\r\nバイオサイエンス   
...                      ...   
1009               エッジテクノロジー   
1010                 ライトワークス   
1011   ビッグツリーテクノロジー＆コンサルティング   
1012                  セイファート   
1013  Recovery International   

                                                    url  code  成長性  割安性  話題性  \
0      https://www.ipokiso.com//company/2011/aisei.html  3170    ◎    〇    〇   
1      https://www.ipokiso.com//company/2011/unico.html  3169    〇    △    〇   
2

In [ ]:
df_result2

,company_name,url,code,成長性,割安性,話題性,総合評価,会社名,コード,市場,会社URL,狙い目証券会社,会社設立,会社名,コード,市場,会社URL,狙い目証券会社,会社設立
0,アイセイ薬局,https://www.ipokiso.com//company/2011/aisei.html,3170,◎,〇,〇,B,アイセイ薬局（3170） 【JASDAQスタンダード】,NaN,NaN,http://www.aisei.co.jp/,SMBC日興証券（副幹事）、SBI証券,NaN,アイセイ薬局（3170） 【JASDAQスタンダード】,NaN,NaN,http://www.aisei.co.jp/,SMBC日興証券（副幹事）、SBI証券,NaN
1,ミサワ,https://www.ipokiso.com//company/2011/unico.html,3169,〇,△,〇,C,ミサワ（3169） 【JASDAQスタンダード】,NaN,NaN,http://www.unico-fan.co.jp/,SMBC日興証券（副幹事）、SBI証券、松井証券,NaN,ミサワ（3169） 【JASDAQスタンダード】,NaN,NaN,http://www.unico-fan.co.jp/,SMBC日興証券（副幹事）、SBI証券、松井証券,NaN
2,スターフライヤー,https://www.ipokiso.com//company/2011/starflye...,9206,〇,◎,〇,B,スターフライヤー（9206） 【東証2部】,NaN,NaN,http://www.starflyer.jp/,SMBC日興証券（副幹事）、マネックス証券 カブドットコム証券,NaN,スターフライヤー（9206） 【東証2部】,NaN,NaN,http://www.starflyer.jp/,SMBC日興証券（副幹事）、マネックス証券 カブドットコム証券,NaN
3,新田ゼラチン,https://www.ipokiso.com//company/2011/nitta-ge...,4977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,カイオム・\r\nバイオサイエンス,https://www.ipokiso.com//company/2011/chiome.html,4583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,エッジテクノロジー,https://www.ipokiso.com//company/2022/edge-tec...,4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,ライトワークス,https://www.ipokiso.com//company/2022/lightwor...,4267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,ビッグツリーテクノロジー＆コンサルティング,https://www.ipokiso.com//company/2022/bigtreet...,4266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012,セイファート,https://www.ipokiso.com//company/2022/seyfert....,9213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [950]:
df_basic_info.to_csv("df_basic_info.csv", encoding='utf-8_sig')

[2]の取得

[3]の取得

[4]の取得

[5]の取得

[6]の取得

[7]の取得

以降参考データ

In [ ]:
df_html_table.T.iloc[1:,:].values

In [806]:
df_html_table = pd.read_html("https://www.ipokiso.com/company/2022/tms-japan.html")[4]

cols = df_html_table.T.iloc[0,:].values
val = df_html_table.T.iloc[1:,:].values.flatten()
df_ipo_stock_info = pd.DataFrame([val],columns=cols)


In [ ]:
val

In [ ]:
# re.subで文字列置換している　(arr.split("株")[i])の文字列のr'/D'に一致したものを''に置換する
# \Dは数字以外の文字列
# splitで株で区切って株自体は消す。配列にいれる
# intで数字に変換
stock_info = [
    int(re.sub( r'\D', '', (arr.split("株")[i]))) 
    for arr in val
    for i in range(3) 
]
# ravelは配列を一次元化する
# ravel()は可能な限りビューを返すが、flatten()は常にコピーを返すという違いがある。
# カラムを２つ国内と海外を作成してリスト化、1次元化している。
cols_1 = np.ravel([[c,c+"_国内",c+"_海外"] for c in cols])
# 分割したデータと、名称を変更したカラムでデータフレームを作成している。
df_stock_info = pd.DataFrame([stock_info],columns=cols_1)

以後チェック用code

In [ ]:
# URLdict（単年）作成 確認用
year = "2020"
url = f'https://www.ipokiso.com/company/{year}.html'
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')
find_all_list = soup.find_all(href=re.compile(f"company/{year}/"))
url_dict = {
    find_all_list[i].text:"https://www.ipokiso.com/" + soup.find_all(href=re.compile(f"company/{year}/"))[i]["href"]
    for i in range(len(find_all_list))
}
url_dict

In [ ]:
# 単年のデータ取得2011~2017　確認用
df_result = pd.DataFrame()
year = "2011"
url =  rf"https://www.ipokiso.com/company/{year}.html"
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')
find_all_list = soup.find_all(href=re.compile(f"company/{year}/"))
print(len(find_all_list))
df_url = pd.read_html(url)
time.sleep(np.random.randint(100,120)/100)
for i in range(len(df_url)):
    df_url[i]["上場年"] = f"{year}"
    df_result = pd.concat([df_result, df_url[i]])
    df_result = df_result.reset_index(drop=True)
df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
df_result

In [ ]:
# 単年のデータ取得2018~2022　確認用
df_result = pd.DataFrame()
year = "2022"
url =  rf"https://www.ipokiso.com/company/{year}.html"
if year == "2022":
    url =  r"https://www.ipokiso.com/company/index.html"
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')
find_all_list = soup.find_all(href=re.compile(f"company/{year}/"))
print(len(find_all_list))
df_url = pd.read_html(url)
time.sleep(np.random.randint(100,120)/100)
for i in range(0, len(df_url) , 2):
    df_url_con = pd.concat([df_url[i], df_url[i+1]], axis=1)
    df_url_con["上場年"] = f"{year}"
    df_result = pd.concat([df_result, df_url_con],ignore_index=True)
    df_result = df_result.reset_index(drop=True)
df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
df_result

In [ ]:
"""ArithmeticErrordf = df.drop_duplicates()
#カラムの重複した行を消す。
df = df[(df['申込日'] != "申込日").values].reset_index(drop=1)
#index番号をリセットする。
col = ['申込日', '株数', '前週比', '金額', '前週比.1', '株数.1', '前週比.1.1', '金額.1', '前週比.1.2','評価損益率(%)', '信用倍率']
df.columns = col
#columns（カラムズ）パラメータで列名を自分で指定し修正する。
#2重になってたカラムも一つに統一できる。MultiIndex⇒Indexにパラメータを変更。
"""

In [ ]:
# df.to_csv(r"C:\Users\xxp2p\OneDrive\ドキュメント\MEGA_saya\Traders_web\margin_transition\csv\margin_transition.csv", encoding='utf-8_sig')
#文字化け改善 encoding='utf-8_sig

In [ ]:
#読み込み
# pd.read_csv(r"C:\Users\xxp2p\OneDrive\ドキュメント\MEGA_saya\Traders_web\margin_transition\csv\margin_transition.csv", header=None, na_values=['-'])

In [ ]:
#Pythonの文字列を拡張するf,r,b,uについて
#f… .format()を使うことなく文字列の中に変数を埋め込むことができる。
#r… raw string　通常はバックスラッシュ（改行）があった場合エスケープシーケンスが働くが、raw stringを用いることで無視。
#b…バイト列リテラル str型ではなくバイト型のインスタンスを作成します。
#u…Unicodeに変換する
#fとrだけ使えればOK！

In [ ]:
#縦方向に結合する方法|append(), concat()  参考URL https://obgynai.com/python-pandas-index-merge-join/
#append：新しい行を追加するメソッド  
#concat：columns(列名)や index (行名)を参照して結合するメソッド。ignore_indexとはindexを無視するという意味。concatでデータフレームを結合した際にindexに付与された番号を無視すると言う事になる。
#drop dropで行を削除する  参考URL　https://www.sejuku.net/blog/73830

In [ ]:
#データのスクレイピングとデータ整形のコードは分ける。